In [ ]:
import instaloader
import pandas
import csv

username = 'HA_Scrapper'
password = 'Disturbuteablepassword'

# Initialize Instaloader
L = instaloader.Instaloader()

# Login to Instagram
L.context.username = username
L.context.password = password
L.login(username, password)

In [ ]:
# Load your CSV of Instagram user IDs into a list
with open('user_ids.csv', 'r') as f:
    reader = csv.reader(f)
    user_ids = list(reader)

In [ ]:
# Dataframes for the required data
df_location = pandas.DataFrame(columns=['insta_id', 'post_shortcode', 'location'])
df_profile_info = pandas.DataFrame(columns=['insta_id', 'is_private', 'follower_count'])

In [ ]:
# Iterate over the Instagram user IDs
for i, user_id in enumerate(user_ids):
    profile = instaloader.Profile.from_id(L.context, int(user_id[0]))

    # Saving profile info
    df_profile_info = df_profile_info.append({'insta_id': user_id[0], 'is_private': profile.is_private, 'follower_count': profile.followers}, ignore_index=True)  # type: ignore
    print(f'Processed {i+1}/{len(user_ids)} profiles. Added {user_id[0]} to profile_info DataFrame.')
    
    # If profile is public
    if not profile.is_private:
        # Get the last 5 posts
        for post in profile.get_posts()[:5]:
            # Check if the post has a location
            if post.location:
                # If location is in India
                if 'India' in post.location.name:
                    df_location = df_location.append({'insta_id': user_id[0], 'post_shortcode': post.shortcode, 'location': post.location.name}, ignore_index=True)  # type: ignore
                    print(f'Added a post from {user_id[0]} with location in India to location_data DataFrame.')

In [ ]:
# Save dataframes to CSV
df_location.to_csv('location_data.csv')
df_profile_info.to_csv('profile_info.csv')